## 引言
前面一篇文章[欺诈文本分类微调（四）：构造训练/测试数据集](https://golfxiao.blog.csdn.net/article/details/141325192)已经构造出了测试数据集，这篇文章将基于测试数据集做模型评测，由于还没有开始训练，所以先对基座模型做评测。

我们的任务目标是对文本进行分类，所以评测的目标是计算模型的精确率和召回率。评测的过程大概是：
1. 用目标模型对每条数据的input做推理，得到一个预测值。
2. 收集所有数据的预测值和标签值，并比较预测正确和预测错误的数量。
3. 根据比较结果计算模型的精确率和召回率。


## 准备数据和模型

先导入需要用到的库，其中：
- transformers用于加载原始模型
- peft用于加载微调模型
- sklearn.metrics用于计算精确率和召回率

In [37]:
import os
import json
import re
import torch
from tqdm import *
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
from sklearn.metrics import confusion_matrix, roc_curve, precision_recall_curve, auc

定义评估数据集所在文件路径，以及原始模型路径

In [2]:
testdata_path = '/data2/anti_fraud/dataset/eval0819.jsonl'
model_path = '/data2/anti_fraud/models/modelscope/hub/Qwen/Qwen2-1___5B-Instruct'
device = 'cuda'

#### 加载数据

上面定义的评估数据集采用jsonl格式保存，所以需要一个方法来加载jsonl格式的数据集，本质上就是用json.loads分别加载每条数据，最后再组成一个list。

In [3]:
def load_jsonl(path):
    with open(path, 'r') as file:
        data = [json.loads(line) for line in file]
        return data

test_data = load_jsonl(testdata_path)

查看下数据集是否均衡。

In [4]:
true_data = [d for d in test_data if d['label'] == True]
false_data = [d for d in test_data if d['label'] == False]
print(f"total_count: {len(test_data)}, true_data: {len(true_data)}, false_data: {len(false_data)}")

total_count: 2348, true_data: 1183, false_data: 1165


#### 加载模型
定义一个方法load_model来同时支持加载原始模型和微调后的模型，使用时的区别在于是否传参微调后的checkpoint_path。

In [6]:
def load_model(model_path, checkpoint_path='', device='cuda'):
    # 加载tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True, padding_side="left")
    # 加载模型
    model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=torch.bfloat16, trust_remote_code=True).eval().to(device)
    # 加载lora权重
    if checkpoint_path: 
        model = PeftModel.from_pretrained(model, model_id=checkpoint_path).to(device)
    
    return model, tokenizer

In [7]:
model, tokenizer = load_model(model_path)
model

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 1536)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear(in_features=1536, out_features=1536, bias=True)
          (k_proj): Linear(in_features=1536, out_features=256, bias=True)
          (v_proj): Linear(in_features=1536, out_features=256, bias=True)
          (o_proj): Linear(in_features=1536, out_features=1536, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (up_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (down_proj): Linear(in_features=8960, out_features=1536, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm()
        (post_attention_layernorm): Qwen2RMSNorm()
      )
    )
    (norm): Qwen2RMSNorm()
  )
  (lm_head): Linear

定义一个推理函数predict，用传入的模型和一条对话文本来进行欺诈文本分类的推理。此函数定义时要考虑以下几点：
1. 既要测试原始模型，也要测试微调后的模型，并且可能会微调多个版本，所以把model和tokenizer作为参数传入。
2. 由于模型预测结果的不确定性，在使用json加载解析response时可能会报异常，需要加一个safe_loads保护。

> 注：语言模型在生成json时，很容易输出markdown文本中嵌入的json格式，所以在解析json之前先尝试去除markdown代码块的头和尾。

In [8]:
def safe_loads(text, default_value=None):
    json_string = re.sub(r'^```json\n(.*)\n```$', r'\1', text.strip(), flags=re.DOTALL)
    try:  
        return json.loads(json_string)  
    except json.JSONDecodeError as e:  
        print(f"invalid json: {json_string}")
        return default_value  

def predict(model, tokenizer, content, device='cuda', debug=False):
    prompt = f"下面是一段对话文本, 请分析对话内容是否有诈骗风险，只以json格式输出你的判断结果(is_fraud: true/false)。\n\n{content}"
    inputs = tokenizer.apply_chat_template([{"role": "user", "content": prompt}],
                                           add_generation_prompt=True,
                                           tokenize=True,
                                           return_tensors="pt",
                                           return_dict=True
                                           ).to(device)
    
    print(f"inputs.shape: {inputs['input_ids'].shape[1]}") if debug else None
    default_response = {'is_fraud':False}
    gen_kwargs = {"max_new_tokens": 2048, "do_sample": True, "top_k": 1}
    with torch.no_grad():
        outputs = model.generate(**inputs, **gen_kwargs)
        print(f"outputs.shape: {outputs.shape}") if debug else None
        outputs = outputs[:, inputs['input_ids'].shape[1]:]
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        return safe_loads(response, default_response)

predict(model, tokenizer, '郎艳: 最近找到了一条新的货源，在重庆那边，如果您愿意投资，可以以很低的价格买到衣服，并且出租赚钱哦。投资金额500元到10000元不等，非常划算的。')

{'is_fraud': False}

#### 评测方法

评测主要是对所有数据的预测结果进行是否正确的统计与分析，最终根据自己的侧重点来计算出一两个指标用于评估模型性能。

首先定义一个方法，对所有的test_data来进行分类预测，并返回所有数据的真实标签和预测标签。

In [38]:
def run_test(model, tokenizer, test_data, device='cuda', debug=False):
    real_labels = []
    pred_labels = []
    pbar = tqdm(total=len(test_data), desc=f'progress')
    
    for i, item in enumerate(test_data):
        dialog_input = item['input']
        real_label = item['label']
        
        prediction = predict(model, tokenizer, dialog_input, device)
        pred_label = prediction['is_fraud']
        
        real_labels.append(real_label)
        pred_labels.append(pred_label)
        
        pbar.update()
        # print(f"percent: {(i*100)/len(test_data):.2f}%") if (debug and i%(len(test_data)//20 + 1)==0) else None
    return real_labels, pred_labels

用一个迷你数据集（只有10条数据）来测试一下。

In [39]:
real_labels, pred_labels = run_test(model, tokenizer, test_data[:10], debug=True)
real_labels, pred_labels

progress: 100%|██████████| 10/10 [00:03<00:00,  3.27it/s]


([False, False, False, True, True, False, True, True, False, False],
 [True, False, False, False, False, False, False, True, False, False])

In [ ]:
对于二分类问题，常用的评测指标是计算精确率和召回率。

定义计算召回率和精确率的方法，相关方法和概念解释如下：
- confusion_matrix函数接受一个标签分类值集合和预测分类值集合，返回一个2x2的矩阵包含4个数据，分别是真负、假正、假负、真正的统计数字。
- precision:精确率，表示预测为正的结果中有多少是真的正值，计算公式为：精确率=真正/(真正+假正）
- recall：召回率，表示标签中为正的数据有多少被成功预测召回，计算公式为：召回率=真正/(真正+假负）                                                 

In [34]:
def precision_recall(true_labels, pred_labels, labels=None, debug=False):
    cm = confusion_matrix(true_labels, pred_labels, labels=labels)
    tn, fp, fn, tp = cm.ravel()
    print(f"tn：{tn}, fp:{fp}, fn:{fn}, tp:{tp}") if debug else None
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    return precision, recall

precision_recall(real_labels, pred_labels, labels=[True, False], debug=True)

tn：1, fp:3, fn:1, tp:5


(0.625, 0.8333333333333334)

> 上面的(0.625, 0.833)就是模型在上面10条数据上计算出的精确率和召回率。

## 运行评测

将上面的步骤封装到一个evaluate方法中，这样只需要一句代码就能运行评估并输出结果。

In [40]:
def evaluate(model_path, checkpoint_path, dataset, device='cuda', debug=False):
    model, tokenizer = load_model(model_path, checkpoint_path, device)
    true_labels, pred_labels = run_test(model, tokenizer, dataset, device, debug=debug)
    precision, recall = precision_recall(true_labels, pred_labels, debug=debug)
    print(f"precision: {precision}, recall: {recall}")

计算原始模型的精确率和召回率

In [41]:
device = 'cuda:3'

#### 原始模型

In [42]:
evaluate(model_path, '', test_data, debug=True)

progress:   5%|▌         | 123/2348 [00:43<31:42,  1.17it/s]

invalid json: 很抱歉，由于我无法直接查看或访问任何外部数据集，因此无法分析对话内容是否包含欺诈行为。我的功能仅限于回答一般性问题和提供信息。如果您有其他相关的问题，请随时提问。


progress:   8%|▊         | 191/2348 [01:06<16:34,  2.17it/s]

invalid json: 很抱歉，由于我无法直接查看或处理文件，请提供具体的对话文本。


progress:  80%|███████▉  | 1874/2348 [08:22<05:00,  1.58it/s]

invalid json: 很抱歉，由于我无法直接查看或访问任何外部数据集，因此无法分析对话内容是否包含欺诈行为。我的功能仅限于回答一般性问题和提供信息。如果您有其他相关的问题，请随时提问。


progress: 100%|██████████| 2348/2348 [10:24<00:00,  3.76it/s]

tn：1095, fp:70, fn:644, tp:539
precision: 0.8850574712643678, recall: 0.4556213017751479


## 封装脚本
由于模型微调往往要反复调参进行训练，每次训练完都需要用上面的过程来评测模型的精确率和召回率指标，所以有必要将上面的评测过程封装为一个`evaluate.py`，以方便使用。

In [ ]:
import os
import json
import re
import torch
from typing import List, Dict
from tqdm import *
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
from sklearn.metrics import confusion_matrix, roc_curve, precision_recall_curve, auc

def load_jsonl(path):
    with open(path, 'r') as file:
        data = [json.loads(line) for line in file]
        return data

def load_model(model_path, checkpoint_path='', device='cuda'):
    # 加载tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
    # 加载模型
    model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=torch.bfloat16, trust_remote_code=True).eval().to(device)
    # 加载lora权重
    if checkpoint_path: 
        model = PeftModel.from_pretrained(model, model_id=checkpoint_path).to(device)
    
    return model, tokenizer

def safe_loads(text, default_value=None):
    json_string = re.sub(r'^```json\n(.*)\n```$', r'\1', text.strip(), flags=re.DOTALL)
    try:  
        return json.loads(json_string)  
    except json.JSONDecodeError as e:  
        print(f"invalid json: {json_string}")
        return default_value  

def predict(model, tokenizer, content, device='cuda', debug=False):
    prompt = f"下面是一段对话文本, 请分析对话内容是否有诈骗风险，只以json格式输出你的判断结果(is_fraud: true/false)。\n\n{content}"
    inputs = tokenizer.apply_chat_template([{"role": "user", "content": prompt}],
                                           add_generation_prompt=True,
                                           tokenize=True,
                                           return_tensors="pt",
                                           return_dict=True
                                           ).to(device)
    
    default_response = {'is_fraud':False}
    gen_kwargs = {"max_new_tokens": 2048, "do_sample": True, "top_k": 1}
    with torch.no_grad():
        outputs = model.generate(**inputs, **gen_kwargs)
        outputs = outputs[:, inputs['input_ids'].shape[1]:]
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        return safe_loads(response, default_response)

def run_test(model, tokenizer, test_data, device='cuda', debug=False):
    real_labels = []
    pred_labels = []
    pbar = tqdm(total=len(test_data), desc=f'progress')
    for i, item in enumerate(test_data):
        dialog_input = item['input']
        real_label = item['label']
        
        prediction = predict(model, tokenizer, dialog_input, device)
        pred_label = prediction['is_fraud']
        
        real_labels.append(real_label)
        pred_labels.append(pred_label)
        pbar.update(1)
    return real_labels, pred_labels

def precision_recall(true_labels, pred_labels, labels=None, debug=False):
    cm = confusion_matrix(true_labels, pred_labels, labels=labels)
    tn, fp, fn, tp = cm.ravel()
    print(f"tn：{tn}, fp:{fp}, fn:{fn}, tp:{tp}") if debug else None
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    return precision, recall

def evaluate_with_model(model, tokenizer, testdata_path, device='cuda', debug=False):
    dataset = load_jsonl(testdata_path)
    run_test_func = run_test_batch if batch else run_test
    true_labels, pred_labels = run_test_func(model, tokenizer, dataset, device=device, debug=debug)
    precision, recall = precision_recall(true_labels, pred_labels, debug=debug)
    print(f"precision: {precision}, recall: {recall}")

def evaluate(model_path, checkpoint_path, testdata_path, device='cuda', debug=False):    
    model, tokenizer = load_model(model_path, checkpoint_path, device)
    evaluate_with_model(model, tokenizer, testdata_path, device, batch, debug)

#### 7B模型评测对比

使用此脚本对Qwen2-7B模型的评测结果。

In [2]:
%run evaluate.py
model_path = '/data2/anti_fraud/models/modelscope/hub/Qwen/Qwen2-7B-Instruct'
evaluate(model_path, '', testdata_path, device, batch=True, debug=True)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

run in batch mode, batch_size=8


progress:   5%|████▏                                                                                    | 112/2348 [00:20<12:40,  2.94it/s]

invalid json: 为了提供准确的判断，我需要更多的上下文信息来分析这段对话。仅凭“发言人3: 明白”这一句话，很难判断是否存在诈骗风险。通常在分析对话时，会考虑对话中的具体语境、涉及的交易细节、请求或提供的信息类型等因素。例如，如果在这之前有提到某个具体的交易、投资机会、汇款请求等，并且这些内容存在不寻常、可疑或可能误导的情况，那么这句“明白”可能会被看作是潜在诈骗的一部分。

由于没有足够的上下文信息，我无法准确判断这段对话是否包含诈骗风险。因此，基于当前的信息，我将答案设置为：

```json
{
  "is_fraud": null
}
```

这意味着根据提供的信息，无法确定是否存在诈骗风险。如果有更多的对话内容或者更详细的背景信息，请提供以便进行更准确的分析。


progress:   5%|████▊                                                                                    | 128/2348 [00:23<10:03,  3.68it/s]

...

invalid json: 为了提供准确的判断，我需要完整的对话上下文。然而，根据提供的片段 "发言人10: 那"，信息量非常有限，不足以确定是否存在诈骗风险。通常分析对话中的诈骗风险会考虑涉及的交易细节、紧急请求、个人信息索取、不寻常的优惠等内容。由于缺少这些关键信息，我无法做出判断。

```json
{
  "is_fraud": null
}
```

请注意，完整对话上下文对于此类分析至关重要。如果能提供更多对话内容，我可以给出更准确的判断。


progress:  97%|█████████████████████████████████████████████████████████████████████████████████████▍  | 2280/2348 [05:19<00:17,  3.84it/s]

invalid json: 为了提供准确的判断，我需要完整的对话内容来进行分析。仅凭“发言人2: 其中。”这一句话无法确定是否存在诈骗风险。通常识别诈骗风险需要考虑对话的整体上下文、涉及的交易细节、紧急请求、个人信息泄露的风险、以及任何不寻常或可疑的行为模式等。

因此，基于提供的信息片段，我无法做出判断。完整的对话内容是必要的。如果必须给出一个示例JSON输出，基于当前信息，结果可能是：

```json
{
  "is_fraud": null
}
```

这表示没有足够的信息来确定是否涉及诈骗。


progress: 100%|████████████████████████████████████████████████████████████████████████████████████████| 2348/2348 [05:27<00:00,  7.17it/s]

tn：1110, fp:55, fn:579, tp:604
precision: 0.9165402124430956, recall: 0.5105663567202029
